In [ ]:
# Create a world box
world = mt.createCube(size=[600, 40, 35], pos=[0, -20, 7.5])

# Create cylinders
cylinders = []
positions = [-20, -15, -10, -5, 0, 5, 10, 15]
for pos in positions:
    cylinder = mt.createCylinder(pos=[pos, 0, 0], radius=0.35, height=0.9, nSegments=32)
    cylinders.append(cylinder)

# Merge all geometries
geom = world
for cyl in cylinders:
    geom = geom + cyl
pg.show(geom, boundaryMarker=True)

In [ ]:
# Create terrain rectangles

for pos in positions:
    terra = mt.createRectangle(start=[pos + 0.35, 0], end=[pos + 9.65, -0.9])
    geom = geom + terra

terraNeg = mt.createRectangle(start=[-30, 0], end=[-19.65, -0.9])
terraPos = mt.createRectangle(start=[24.65, 0], end=[30, -0.9])
geom = geom + terraNeg + terraPos

In [ ]:
#tentativa de adicionar marcadores de região
    """

    #geom.addRegionMarker(top_boundaries, 3)
    #geom.addRegionMarker(bottom_boundaries, 4)
    #geom.addRegionMarker(left_boundaries, 5)
    #geom.addRegionMarker(right_boundaries, 6)
    #geom.addRegionMarker(side_boundaries, 7)
    #geom.addRegionMarker(world_boundaries, -2)

    geom.addRegionMarker(pos(bottom_boundaries[0]), 2)
    geom.addRegionMarker(pos(top_boundaries[0]), 0)
    geom.addRegionMarker(pos(side_boundaries[0]), 3)
    geom.addRegionMarker(pos(zero_boundaries[0]), -2) """


#tentativa de criar marcadores
    i = 0
    nodes = [pg.core.Node(coord[1], coord[2]) for coord in coord]
    line_segments = []

    for a in range(len(nodes) - 1):
        start_node = nodes[a]
        end_node = nodes[a + 1]

        if i == 0:
            marker = i + 1
            i += 1
        elif i == 1:
            marker = i + 2
            i += 1
        elif i == 2:
            marker = i + 3
            i += 1
        elif i == 3:
            marker = i + 4
            i = 0

        line_segment = mt.createLine(start=start_node.pos(), end=end_node.pos(), marker=marker)
        line_segments.append(line_segment)

    print('line_segments: ' + str(line_segments))
    pg.show( geom, label='Line segments', markers=True, figsize=(15, 8), orientation='vertical' )


In [ ]:
print(bottom_boundaries[0])

In [ ]:
""" 
# OK
# Calculo doido de coordenadas dos nós
leftneg_boundaries = [boundary.id() for boundary in geom.boundaries() if (boundary.center()[0] < 0 and str(boundary.center()[0]).endswith(esq))]
leftpos_boundaries = [boundary.id() for boundary in geom.boundaries() if (boundary.center()[0] > 0 and str(boundary.center()[0]).endswith(dir))]
rightpos_boundaries = [boundary.id() for boundary in geom.boundaries() if (boundary.center()[0] > 0 and str(boundary.center()[0]).endswith(esq))]
rightneg_boundaries = [boundary.id() for boundary in geom.boundaries() if (boundary.center()[0] < 0 and str(boundary.center()[0]).endswith(dir))]
left_boundaries = leftneg_boundaries + leftpos_boundaries
right_boundaries = rightneg_boundaries + rightpos_boundaries

top_boundaries = [boundary.id() for boundary in geom.boundaries() if boundary.center()[1] == altura_poco]
bottom_boundaries = [boundary.id() for boundary in geom.boundaries() if boundary.center()[1] ==prof_poco]
world_boundaries = [boundary.id() for boundary in geom.boundaries() if boundary.id() not in top_boundaries + bottom_boundaries + left_boundaries + right_boundaries]

print('left_boundaries: ' + str(left_boundaries))
print('bottom_boundaries: ' + str(bottom_boundaries))
print('right_boundaries: ' + str(right_boundaries))
print('top_boundaries: ' + str(top_boundaries))
print('world_boundaries: ' + str(world_boundaries))


def pos(count):
    return pg.pos(node[count][1], node[count][2])
pg.show(geom, label='Boundary marker', showNodes=True,markers=True, showMesh=True, figsize=(15, 8), orientation='vertical' ) """

In [ ]:
Teste modelo de esvaziamento do poço
""" # Definições de variáveis e funções
#h0 = altura_poco            # Altura inicial da água no poço
#h2o = h0 - prof_poco        # Altura da linha da água inicial no poço
#h2o2=0
#y=0.0001
#y = 0.1  # Defina a variação na linha vertical da fronteira
# Defina os parâmetros iniciais hidráulicos
#taxa_de_infiltracao = 0.005 # Taxa de infiltração em m/s
#num_iteracoes = 20          # Número de iterações dt para simulação
#dt = 1                      # Intervalo de tempo em segundos
 """

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import pygimli as pg

# Definições de variáveis e funções
r = 0.35  # Raio do poço em metros
h0 = 0.9  # Altura inicial da água no poço em metros
A_bottom = np.pi * r**2  # Área da base do poço em metros quadrados

h202 = h0  # Altura inicial da água no poço em metros

dt = 1  # Intervalo de tempo em segundos
t_final = 3600  # Tempo final da simulação em segundos

L_lateral = h0  # Comprimento da área de vazamento lateral em metros

k_lateral = 7e-4  # Taxa de vazamento lateral em metros quadrados por segundo
k_bottom = k_lateral  # Taxa de vazamento de fundo em metros quadrados por segundo

g = 9.81  # Aceleração da gravidade em metros por segundo quadrado

# Coeficiente de dispersão (simplificado, constante)
D = 0.0001  # metros quadrados por segundo

# Número de segmentos radiais para a simulação de advecção-dispersão
nr = 10
dr = r / nr
rad = dr/2 + dr * np.arange(nr)  # Posições radiais

# Inicialização
t = np.arange(0, t_final + dt, dt)
h = np.zeros(len(t))
h[0] = h0
conc = np.zeros((nr, len(t)))  # Concentração em relação ao raio
conc[:, 0] = 1  # Concentração inicial é 1

# Crie a figura e a animação
fig, ax = plt.subplots()
ax.set_aspect('equal')
ax.set_xlim(-r, r)
ax.set_ylim(0, h0)
ax.set_xlabel('X (m)')
ax.set_ylabel('Y (m)')
ax.set_title('Seção transversal da simulação de drenagem do poço (animada)')
ax.grid(True)
# Inicialize o patch da água (aproximação retangular para o círculo)
water_patch = plt.Rectangle((-r, 0), 2*r, h[0], color='b')
# Adicione um texto para exibir o tempo atual
time_text = ax.text(-0.3*r, h0 - 0.3, 'Tempo: 0 s', fontsize=12, color='k')
# Adicione o patch da água e o texto ao eixo
ax.add_patch(water_patch)
ax.add_artist(time_text)
# Loop para cada passo de tempo
for i in range(len(t) - 1):
  # Cálculo do vazamento lateral
  Q_lateral_total = 0
  h_int_limit = min(h[i], L_lateral)
  if h_int_limit > 0:
    Q_lateral_total = k_lateral * 2 * np.pi * r * h[i]

  # Cálculo do vazamento de fundo
  Q_bottom = k_bottom * np.pi * r**2 * h[i]

  # Vazamento total
  Q_total = Q_lateral_total + Q_bottom

  # Cálculo da variação da altura da água
  dh = - (Q_total / A_bottom) * dt

  # Atualize a altura da água
  h[i + 1] = h[i] + dh

  # Previna altura da água negativa
  if h[i + 1] < 0:
    h[i + 1] = 0
    break

  # Simulação de advecção-dispersão
  for j in range(nr):
    # Advecção (simplificada como uma fração do vazamento total)
    adv = - (Q_total / (A_bottom * h0)) * conc[j, i] * dt

    # Difusão (simplificada como difusão radial)
    if j > 0 and j < nr - 1:
      diff = D * (conc[j + 1, i] - 2 * conc[j, i] + conc[j - 1, i]) / (dr**2) * dt
    elif j == 0:
      diff = D * (conc[j + 1, i] - conc[j, i]) / (dr**2) * dt  # condição de contorno sem fluxo em r=0
    elif j == nr - 1:
      diff = D * (-conc[j, i] + conc[j - 1, i]) / (dr**2) * dt  # condição de contorno de escoamento

    conc[j, i + 1] = conc[j, i] + adv + diff

    # Previna concentração negativa
    if conc[j, i + 1] < 0:
      conc[j, i + 1] = 0

  # Atualize o patch da água para a animação
  water_patch.set_height(h[i + 1])

  # Atualize o texto do tempo
  time_text.set_text(f'Tempo: {t[i + 1]:.0f} s')

  # Desenhe a figura
  plt.draw()
  plt.pause(0.005)

# Crie a figura para o gráfico
fig2 = plt.figure()

# Crie o eixo para o gráfico
ax2 = fig

In [ ]:
""" PANICO """
#parcial = 10
#mostrar_parcial = False
#mostrar_cada_iteracao = False

#p = np.random.rand(mesh.cellCount())                   # Inicialize com potencial hidráulico aleatorio


# Resolver o modelo de potencial hidráulico
p = pg.solver.solveFiniteElements(mesh, a=K, bc=pBound, verbose=True)


#Loop para simular o preenchimento e esvaziamento do poço 
# 
# 
# ## já vai ser feito lá embaixo em FiniteVolume  
#   ainda vai precisar simular esvaziamento pras fronteiras laterais do poço
# 
# Calcular a nova altura do poço com base na taxa de esvaziamento
#h2o = h0 - prof_poco - taxa_de_infiltracao * dt
# Garantir que a altura do poço não fique negativa


#    if mostrar_cada_iteracao == True:
#        pg.show(mesh, data=p, label='Potencial Hidráulico após iteração {}'.format(t))
#    if mostrar_parcial == True:
#        if t % parcial == 0:
#            pg.show(mesh, data=p, label='Potencial Hidráulico após iteração {}'.format(t))




# (Opcional) Adicionar uma pausa para visualização
# plt.pause(0.1)  # Se você estiver usando matplotlib para visualização
#pg.show(mesh, rhoa=p, label='Potencial Hidráulico após iteração {}'.format(t))
#print(pBound)
#print(h2o)

# OK
# Calculate statistics
mean_p = np.mean(p)
std_p = np.std(p)
min_p = np.min(p)
max_p = np.max(p)

print(f"Mean: {mean_p}, Std: {std_p}, Min: {min_p}, Max: {max_p}")


In [ ]:
#Tentativa Calculate the advection-dispersion term with dependency on height (y)
#Tentativa de advection
    for boundary_id in zero_boundaries:
        grad_c = np.gradient(p)  # Gradient of hydraulic potential
        div_grad_c = np.gradient(grad_c) + np.gradient(grad_c)  # Divergence of the gradient
        y = mesh.boundary(boundary_id).center()[1]  # Get the y-coordinate of the boundary center
        if len(grad_c) > 1 and boundary_id < len(grad_c[0]):  # Ensure grad_c has enough elements and boundary_id is within range
            #advection_dispersion = phi * p / dt - phi * D * div_grad_c - K * grad_c[0][boundary_id] - y
            #pBound["Robin"][boundary_id] = advection_dispersion
#tentativa
    for boundary_id in zero_boundaries:  # Calculate the advection-dispersion term
        grad_c = np.gradient(p)  # Gradient of hydraulic potential
        div_grad_c = np.gradient(grad_c) + np.gradient(grad_c)  # Divergence of the gradient
        if len(grad_c) > 1 and boundary_id < len(grad_c[0]):  # Ensure grad_c has enough elements and boundary_id is within range
            advection_dispersion = phi * p / dt - phi * D * div_grad_c - K * grad_c[0][boundary_id]
            pBound["Robin"][boundary_id] = advection_dispersion

#Tentativa h2o2
    for boundary_id in side_boundaries:
        boundary = mesh.boundary(boundary_id)
        if boundary is not None:
            y = boundary.center()[1]  # Get the y-coordinate of the boundary center
            h2o2 = h2o - y  # Calculate the water height at the boundary
            pBound["Neumann"][boundary_id] = h2o2  # Atualize a altura da água


# condiçoes de fronteira erradas
    for boundary_id in top_boundaries:
        pBound["Dirichlet"][boundary_id] = 0


    for boundary_id in bottom_boundaries:
        pBound["Neumann"][boundary_id] = h2o  # Atualize a altura da água

    # Debug print to check pBound contents
    #print("pBound:", pBound)

In [ ]:
#pg.show(geom, markers=True, nodes=True, boundaries=True)
for b in mesh.boundaries():
    if b.marker() == 0 and not b.outside():        # Se a fronteira é um furo
        b.setMarker(5)
    if b.marker() !=5:
        print(f"Boundary ID: {b.id()}, Marker: {b.marker()}")

# condicoes de fronteira corretas
for b in mesh.boundaries():
    if b.marker() == -2:                            # Se a fronteira é o mundo inferior
        pBound["Dirichlet"][b.id()] = 1

    if b.marker() == 1:                             # Se a fronteira é superior
        pBound["Dirichlet"][b.id()] = 0

    #print(mesh)
    #ax, _ = pg.show(mesh, markers=True, showMesh=True)                         # Mostrar a malha



# Debug print to check pBound contents
if -2 in pBound.get("Dirichlet", {}):
    print("pBound:", pBound)


In [ ]:

# Debug print to check pBound contents
print("pBound:", pBound)

In [ ]:

S = pg.RVector(len(Source_vector))  # Initialize S as a zero vector


#Teste modelo de esvaziamento do poço
""" # Definições de variáveis e funções
#h0 = altura_poco            # Altura inicial da água no poço
#h2o = h0 - prof_poco        # Altura da linha da água inicial no poço
#h2o2=0
#y=0.0001
#y = 0.1  # Defina a variação na linha vertical da fronteira
# Defina os parâmetros iniciais hidráulicos
#taxa_de_infiltracao = 0.005 # Taxa de infiltração em m/s
#num_iteracoes = 20          # Número de iterações dt para simulação
#dt = 1                      # Intervalo de tempo em segundos
"""
 
 
#Or "Matlab" code
 
 
 
 

# Loop for each time step
i=0
for i in enumerate(t):
    # Calculate new source vector with decay (adjust decay function as needed)
    S = (Source_vector * ((0/cone))**i) # Usar 1.0 para garantir divisão de ponto flutuante
    i+=1
    c1 = pg.solver.solveFiniteVolume(mesh, a=dispersion, f=S, vel=veln, times=t, scheme='PS', verbose=0)


# Solve without injection after the injection period
c2 = pg.solver.solveFiniteVolume(mesh, a=dispersion, f=0, vel=veln, u0=c1[-1], times=t, scheme='PS', verbose=0)

In [ ]:
""" mm_stom_s=1e-3 
ltom3=1e-3
#prism=raio_poco*raio_poco
cil_ret=raio_poco*3.14/2
vol_extrd=cil_ret

Source_vector = pg.RVector(mesh.cellCount(),0)
def atualiza_concentracao_salina(ti, a, bores, mesh, prof_poco, altura_poco, raio_poco, TracerCon, ltom3, vol_extrd, Source_vector):
    

    # Defina a altura móvel para este passo de tempo
    altura_movel = 0.000000003*ti**(2)-0.0017*ti+0.9  # Aproximação da altura móvel que diminui com o tempo

    # Iterando sobre os poços
    for pos in bores:
        for y in np.linspace(prof_poco, altura_poco, num=100):
            sourceCelly = mesh.findCell([pos, y])
            if sourceCelly:
                # Atualiza o fluido da celula se não foi feito
                if Source_vector[sourceCelly.id()] == 0:  # Acessa a primeira coluna
                    Source_vector[sourceCelly.id()] = TracerCon * ltom3 / (sourceCelly.size() * vol_extrd)  # g/l * 1e3 / m^3
                
                for x in np.linspace(pos - raio_poco, pos + raio_poco, num=100):
                    sourceCellx = mesh.findCell([x, y])
                    if sourceCellx:  # confirma se esta na célula e atualiza concentraçaõ para a célula 
                        Source_vector[sourceCellx.id()] = TracerCon * ltom3 / (sourceCellx.size() * vol_extrd)  # g/m^3

    # Plota a concentração salina a cada 1000 passos de tempo
    if ti % 1000 == 0:
        pg.show(mesh, data=Source_vector,
                label='Saturação(cincentração/área da célula) salina da célula g/m', cMap='terrain_r', hold=True, showStats=True)

    # Zera valores acima da altura da coluna d'água
    for cell_id in range(mesh.cellCount()):
        cell = mesh.findCell(cell_id)  # Obtém a célula correspondente
        if cell.center()[1] > altura_movel:  # Verifica se a célula está acima da altura móvel
            Source_vector[cell_id] = 0  # Zera o valor no vetor de fonte

# Exemplo de uso:
for ti, a in enumerate(((t))):   
    atualiza_concentracao_salina(ti, a, bores, mesh,prof_poco, altura_poco, raio_poco, TracerCon, ltom3,vol_extrd, Source_vector)
plt.show() """

In [ ]:
\begin{split}\frac{\partial u}{\partial t} + \mathbf{v}\cdot\nabla u & = \nabla\cdot(a \nabla u) + b u + f(\mathbf{r},t)\\
u(\mathbf{r}, t) & = u_B  \quad\mathbf{r}\in\Gamma_{\text{Dirichlet}}\\
\frac{\partial u(\mathbf{r}, t)}{\partial \mathbf{n}} & = u_{\partial \text{B}}  \quad\mathbf{r}\in\Gamma_{\text{Neumann}}\\
u(\mathbf{r}, t=0) & = u_0 \quad\text{with} \quad\mathbf{r}\in\Omega\end{split}

Parameters
:
mesh (GIMLI::Mesh) – Mesh represents spatial discretization of the calculation domain

a (value | array | callable(cell, userData)) – Stiffness weighting per cell values.

b (value | array | callable(cell, userData)) – Scale for mass values b

f (iterable(cell)) – Load vector

fn (iterable(cell)) – TODO What is fn

vel (ndarray (N,dim) | RMatrix(N,dim)) – Velocity field 
v(r,t=const)={[vi]j,}
 with 
i=[1…3]
 for the mesh dimension and 
j=[0…N−1]
 with N either the amount of cells, nodes, or boundaries. Velocities per boundary are preferred and will be interpolated on demand.

u0 (value | array | callable(cell, userData)) – Starting field

times (iterable) – Time steps to calculate for.

Workspace (ws) –

This can be an empty class that will used as an Workspace to store and cache data.

If ws is given: The system matrix is taken from ws or calculated once and stored in ws for further usage.

The system matrix is cached in this Workspace as ws.S The LinearSolver with the factorized matrix is cached in this Workspace as ws.solver The rhs vector is only stored in this Workspace as ws.rhs

scheme (str [CDS]) – Finite volume scheme: pygimli.solver.diffusionConvectionKernel

**kwargs –

bc : Boundary Conditions dictionary, see pg.solver

In [ ]:
# Teste de visualização ERT 
# Assuming DD12 corresponds to a depth of -12 (replace with the actual depth value)
depth_threshold = -12

# Initialize ERTManager
ERT = ERTManager(verbose=False)

# Run simulation for the apparent resistivities
rhoa = ERT.simulate(meshERT, res=resis, scheme=ertScheme, returnArray=True, verbose=False)

# Filter the data below the depth threshold
filtered_rhoa = []
for i in range(len(rhoa)):
    mask = [ertScheme.sensorPosition(ertScheme('a')[j])[1] >= depth_threshold and
            ertScheme.sensorPosition(ertScheme('b')[j])[1] >= depth_threshold and
            ertScheme.sensorPosition(ertScheme('m')[j])[1] >= depth_threshold and
            ertScheme.sensorPosition(ertScheme('n')[j])[1] >= depth_threshold
            for j in range(ertScheme.size())]
    filtered_rhoa.append(np.array(rhoa[i])[mask])

# Visualize the filtered data
for i in range(4):
    ERT.showData(ertScheme, vals=filtered_rhoa[i], cMin=1e-4, cMax=1,)

In [ ]:
# Filter measurements based on a depth threshold
depth_threshold = -12  # Define depth cutoff at -12m

ERT = ERTManager(verbose=False)

# Run simulation for the apparent resistivities
rhoa = ERT.simulate(meshERT, res=resis, scheme=ertScheme, returnArray=True, verbose=False)

for i in range(4):
    # Create a mask for measurements above the depth threshold
    mask = [ertScheme.sensorPosition(ertScheme('a')[j])[1] >= depth_threshold and
           ertScheme.sensorPosition(ertScheme('b')[j])[1] >= depth_threshold and
           ertScheme.sensorPosition(ertScheme('m')[j])[1] >= depth_threshold and
           ertScheme.sensorPosition(ertScheme('n')[j])[1] >= depth_threshold
           for j in range(ertScheme.size())]
    
    # Apply the mask to the data
    filtered_data = rhoa[i][mask]/rhoa[0][mask]
    
    # Create a new scheme with only the filtered measurements
    filtered_scheme = ertScheme.filter(mask)
    
    # Show the filtered data
    ERT.showData(filtered_scheme, vals=filtered_data, logScale=False, 
                 label='Apparent resistivity (filtered)')

In [ ]:
ERT = ERTManager(verbose=False)

# Run simulation for the apparent resistivities
d = ERT.simulate(meshERT, res=resis, scheme=ertScheme, 
                    returnArray=True, verbose=False, )#noiseLevel=1, noiseAbs=1e-3,)

# Filter data to include only values with Y less than 12
mask = np.array([pg.y(ertScheme.sensorPosition(ertScheme('a')[i])) < 12 and
                 pg.y(ertScheme.sensorPosition(ertScheme('b')[i])) < 12 and
                 pg.y(ertScheme.sensorPosition(ertScheme('m')[i])) < 12 and
                 pg.y(ertScheme.sensorPosition(ertScheme('n')[i])) < 12
                 for i in range(ertScheme.size())])
filtered_data = d[:, mask]

# Create a new mesh from the filtered cells
filtered_cells = [cell for cell, m in zip(meshERT.cells(), mask) if m]
filtered_mesh = pg.Mesh(dim=2)
nodes = []
for cell in filtered_cells:
    for node in cell.nodes():
        if node not in nodes:
            nodes.append(node)
            filtered_mesh.createNode(node.pos())
    filtered_mesh.createCell([filtered_mesh.findNearestNode(n.pos()) for n in cell.nodes()])
# Assign the filtered data to the new mesh
filtered_mesh['d'] = filtered_data

#pg.show(filtered_mesh, figsize=(6,4), data=filtered_data,orientation='vertical')          

                    
# Plot the first 4 timesteps normalized by the first timestep
for i in range(4):
    # Create a copy of the scheme for each visualization
    tempScheme = ertScheme.copy()
    tempScheme.set('rhoa', filtered_data[i])
    ERT.showData(tempScheme, vals=filtered_data[i]/filtered_data[0], cMin=1e-4, cMax=1)